In [11]:
import numpy as np
from tqdm import tqdm
import time
import math
import gc
import torch
from datasets import load_dataset
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast, DataCollatorForLanguageModeling

In [119]:
from importlib import reload
import stationary_reversal as sr
reload(sr)

<module 'stationary_reversal' from '/Users/alexinf/Dropbox/github/reverse-dynamics-nlp/reverse-llm-benchmarking/stationary_reversal.py'>

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPTNeoXForCausalLM.from_pretrained(
          "EleutherAI/pythia-70m-deduped-v0"
      ).to(device)
tokenizer = GPTNeoXTokenizerFast.from_pretrained("EleutherAI/gpt-neox-20b")
prefix_length  = 1
suffix = " Obama"
tokenized_suffix= tokenizer.encode(suffix, return_tensors="pt").to(device)
# tokenized_suffix = tokenized_suffix.unsqueeze(0)
suffix_length = len(tokenized_suffix[0])
empirical_dist = torch.load("../data/pile10k_empirical.pt")
vocab_size = empirical_dist.shape[0]

In [121]:
Obama_log_probs_batch = sr.stationary_reverse_full_dist(
  model, empirical_dist, prefix_length, tokenized_suffix)

i= 0


100%|██████████| 32/32 [01:13<00:00,  2.30s/it]
/Users/alexinf/Dropbox/github/reverse-dynamics-nlp/reverse-llm-benchmarking/stationary_reversal.py:126: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [101]:
tokenized_suffix.shape

torch.Size([1, 1])

In [ ]:
reverse_model = GPTNeoXForCausalLM.from_pretrained(
  "afterless/reverse-pythia-160m"
).to(device)
reverse_model.eval()
rev_out = reverse_model(input_ids=tokenized_suffix)
rev_logits = rev_out.logits
rev_logprobs = torch.nn.functional.log_softmax(rev_logits, dim=-1)
rev_probs = torch.exp(rev_logprobs)

In [38]:
Obama_probs = torch.exp(Obama_log_probs_batch)
stat_total_variation = 0.5*torch.sum(torch.abs(Obama_probs[0,:] - rev_probs[0,0,:]))

In [39]:
uniform_dist = (1.0/vocab_size)+torch.zeros(vocab_size)
unif_total_variation = 0.5*torch.sum(torch.abs(uniform_dist - rev_probs[0,0,:]))

In [122]:
from importlib import reload
import stationary_reversal as sr
reload(sr)

tokenized_suffix_test = torch.cat((torch.zeros([1,2], dtype=torch.int32) ,tokenized_suffix),dim=-1)
test_suffix = sr.stationary_reverse_full_dist_suffix_calculation(
      model,
      empirical_dist, 
      tokenized_suffix_test,
      vocab_batch_size=1572)

TypeError: stationary_reverse_full_dist_suffix_calculation() got an unexpected keyword argument 'batch_size'

In [96]:
tokenized_suffix_test.shape

torch.Size([1, 4])

In [83]:
0.5*(torch.sum(torch.abs(torch.exp(test_suffix)-Obama_probs)))

tensor(0.)

In [98]:
test_suffix.shape

torch.Size([3, 50304])